# Домашняя работа 2. Коллаборативная фильтрация (USER-BASED)

Используем ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Решением будет ссылка на гитхаб с готовым ноутбуком

## 1. Импорт библиотек

In [296]:
import pandas as pd
import numpy as np

# Пакет Surprise
from surprise import KNNWithMeans, KNNBasic, NMF
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV


%matplotlib inline

## 2. Загрузка данных

In [297]:
movies = pd.read_csv('../data/ml-1m/movies.dat', sep='::', names=['movieId', 'title', 'genres'])
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])

/Users/akirichenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/Users/akirichenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [298]:
movies = pd.read_csv('../data/ml-latest-small/movies.csv', )
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv',)

In [286]:
movies_with_ratings = ratings.merge(movies, on='movieId', how='left')
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.movieId,
    'rating': movies_with_ratings.rating
})

In [287]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [288]:
# Установим параметры алгоритма
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [289]:
trainset, testset = train_test_split(data, test_size=.25)

In [162]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8818
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8875
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8831
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8829
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8774


In [ ]:
algo.fit(trainset)

In [299]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

In [291]:
predictions = algo.test(testset)

In [292]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8622


0.8621986356275212

In [ ]:
# Use grid search for model selecting:
ks = [30, 50]
min_ks = [5, 10]
param_grid = {'k': ks, 'min_k': min_ks, 
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }

gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

### 3. Cформируем предложения для конкретного пользователя

In [293]:
uid=157
# Подбираем 20 фильмов, которые могут понравиться пользователю
def get_20_personal_predistions(uid):
    df = pd.DataFrame(columns=['userId', 'movieId', 'title', 'rating_pred'])
    uid_movies = movies_with_ratings[(movies_with_ratings.userId==uid)].movieId.values
    iid_movies = [i for i in movies_with_ratings.movieId.unique() if i not in uid_movies]
    for i in iid_movies:
        if trainset.knows_item(i):
            if len(movies[(movies.movieId==i)])!=0:
                title = ''.join(movies[(movies.movieId==i)].title.values)
                prediction = algo.predict(uid=uid, iid=i)
                df = df.append({'userId': uid, 'movieId': i, 'title':title, 'rating_pred': prediction.est}, ignore_index=True)
    return df.sort_values('rating_pred', ascending=False).head(20)

get_20_personal_predistions(uid)

,userId,movieId,title,rating_pred
2952,157,3233,Smashing Time (1967),5.000000
3014,157,3656,Lured (1947),5.000000
2954,157,3172,Ulysses (Ulisse) (1954),5.000000
1986,157,787,"Gate of Heavenly Peace, The (1995)",5.000000
2934,157,1830,Follow the Bitch (1998),5.000000
2904,157,989,Schlafes Bruder (Brother of Sleep) (1995),5.000000
2602,157,3232,Seven Chances (1925),5.000000
2891,157,557,Mamma Roma (1962),5.000000
2511,157,578,"Hour of the Pig, The (1993)",5.000000
3025,157,3607,One Little Indian (1973),5.000000
